In [2]:
import os
import json
import pandas as pd
import re
import string
import nltk
from tqdm import tqdm
from PIL import Image

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
json_dir = "../data/raw/"
files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

In [5]:
data = []
for file in files:
    with open(os.path.join(json_dir, file), encoding='utf-8') as f:
        posts = json.load(f)
        data.extend(posts)

In [6]:
df = pd.DataFrame(data)
df.head()

,id,title,text,image_path,upvotes,created_utc,subreddit,url
0,1l0n4qj,"Strong glance in blue, Romain Eugene, Oils, 2025",,images\1l0n4qj.jpg,609,2025-06-01T11:54:01+00:00,Art,https://i.redd.it/r2f08uf81b4f1.jpeg
1,1l0og4f,"Szarik, NuCatArt (my wife), acrylic painting o...",,images\1l0og4f.jpg,77,2025-06-01T13:02:44+00:00,Art,https://i.redd.it/qi5ut4mldb4f1.jpeg
2,1l0o7fo,"Bondi Beach, Sally West, Oil on Canvas, 2024",,images\1l0o7fo.jpg,73,2025-06-01T12:50:58+00:00,Art,https://i.redd.it/hxta850ibb4f1.jpeg
3,1l0ol4v,"Pri-demon-th 2025, babbittybabbitt, Copic, 2025",,images\1l0ol4v.jpg,67,2025-06-01T13:09:36+00:00,Art,https://i.redd.it/t3dmgyxteb4f1.png
4,1l0j3ca,"Old Money Girl IV, Sculptylife, digital print,...",,images\1l0j3ca.jpg,149,2025-06-01T07:26:42+00:00,Art,https://i.redd.it/7ss3mx8np94f1.jpeg


In [7]:
def clean_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r"http\S+", "", text) 
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text.strip()

df["full_text"] = df["title"].fillna("") + " " + df["text"].fillna("")
df["clean_text"] = df["full_text"].apply(clean_text)

In [8]:
df.drop_duplicates(subset="id", inplace=True)

df = df[df["clean_text"].str.strip() != ""]
df = df[df["image_path"].notna()]

In [9]:
def is_valid_image(path):
    full_path = os.path.join("data/raw/", path)
    try:
        img = Image.open(full_path)
        img.verify()  # Проверка
        return True
    except:
        return False

df["valid_image"] = df["image_path"].apply(is_valid_image)
df = df[df["valid_image"] == True]


In [10]:
for path in tqdm(df["image_path"]):
    full_path = os.path.join("data/raw/", path)
    try:
        img = Image.open(full_path)
        img = img.resize((640, 640))
        img.save(full_path)
    except:
        continue

0it [00:00, ?it/s]


In [12]:
df.to_csv("../data/processed/clean_data.csv", index=False)